In [172]:
import pandas as pd
import numpy as np
from scipy import stats as spstats

from sklearn import model_selection as skms

import tensorflow as tf

import plotly.express as px

In [173]:
heart_data = pd.read_csv('heart.csv')
heart_data.info()
display(heart_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [174]:
heart_data.rename(columns = {'output': 'target'}, inplace = True)
heart_data.info()
display(heart_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [212]:
# Create training, validation, and testing datasets
X = heart_data.drop(columns = 'target')
y = heart_data['target']
split_ratios = {
    'train': 0.70,
    'validation': 0.2,
    'test': 0.1
}

X_train, X_test, y_train, y_test = skms.train_test_split(X, y, train_size = split_ratios['train'])
X_val, X_test, y_val, y_test = skms.train_test_split(X_test, y_test, train_size = split_ratios['validation'] / (1 - split_ratios['train']))

# Convert dataframe/series into tf datasets

# Printing function to look at dataset
# for features_tensor, target_tensor in train_dataset:
#     print(f'features: {features_tensor} target: {target_tensor}')


def dataframe_to_dataset(dataframe, batch_size = 32):
    dataframe = dataframe.copy()
    targets = dataframe.pop('target')
    dataset = tf.data.Dataset.from_tensor_slices((dataframe.values, targets.values)).batch(batch_size)
    return dataset

# train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val.values, y_val.values))
train_dataset = dataframe_to_dataset(X_train.merge(y_train, left_index = True, right_index = True), 10)
val_dataset = dataframe_to_dataset(X_val.merge(y_val, left_index = True, right_index = True), 10)


In [238]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape = (13, )),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer = 'adam', loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics = ['accuracy'])
    return model

model = get_compiled_model()
history = model.fit(train_dataset, epochs = 60, validation_data = val_dataset)


Epoch 1/60
22/22 [==============================] - 0s 4ms/step - loss: 2.4614 - accuracy: 0.4340 - val_loss: 1.6108 - val_accuracy: 0.5000
Epoch 2/60
22/22 [==============================] - 0s 725us/step - loss: 1.1919 - accuracy: 0.4528 - val_loss: 0.9848 - val_accuracy: 0.5833
Epoch 3/60
22/22 [==============================] - 0s 725us/step - loss: 0.8425 - accuracy: 0.5708 - val_loss: 0.8842 - val_accuracy: 0.6667
Epoch 4/60
22/22 [==============================] - 0s 680us/step - loss: 0.7406 - accuracy: 0.6226 - val_loss: 0.8515 - val_accuracy: 0.6833
Epoch 5/60
22/22 [==============================] - 0s 725us/step - loss: 0.6874 - accuracy: 0.6557 - val_loss: 0.8249 - val_accuracy: 0.6833
Epoch 6/60
22/22 [==============================] - 0s 749us/step - loss: 0.6558 - accuracy: 0.6698 - val_loss: 0.7953 - val_accuracy: 0.7167
Epoch 7/60
22/22 [==============================] - 0s 771us/step - loss: 0.6324 - accuracy: 0.6934 - val_loss: 0.7573 - val_accuracy: 0.7333
Epoch 8/

In [239]:
# Checkout out the history
print(history.)

# Evaluate the model on our test data
test_dataset = dataframe_to_dataset(X_test.merge(y_test, left_index = True, right_index = True), 10)
test_results = model.evaluate(test_dataset, verbose = 1)
print(test_results)

4/4 [==============================] - 0s 748us/step - loss: 0.4950 - accuracy: 0.7742
[0.49502986669540405, 0.774193525314331]
